In [4]:
# Boltzmann Machines

# Importing the libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
import os

 #latin because some movie titles have symbols
movies = pd.read_csv('ml-1m/movies.dat', sep= '::', header=None, engine='python', encoding='latin-1')
users = pd.read_csv('ml-1m/users.dat', sep= '::', header=None, engine='python', encoding='latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep= '::', header=None, engine='python', encoding='latin-1')

training_set = np.array(pd.read_csv('ml-100k/u1.base', delimiter='\t'), dtype='int')
test_set = np.array(pd.read_csv('ml-100k/u1.test', delimiter='\t'), dtype='int')

 # we the training set or test set could have the max so we get max of either one
nb_users = int(max(max(training_set[:, 0]), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

# converting the data into an array with users as rows and movies as columns
def convert(data):
    new_data = []
    for id_users in range(0,nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users] # get the list of movies for this user
        id_ratings = data[:,2][data[:,0] == id_users] # get the list of movies ratings for this user
        ratings = np.zeros(nb_movies) # create the ratings array
        ratings[id_movies - 1] = id_ratings # associate the rating with the movie
        new_data.append(list(ratings)) # put each list of ratings in the list
    return new_data

training_set = convert(training_set)
test_set = convert(test_set)

# converting the data into tourch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

# change all the review into binary outputs
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1

test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

In [27]:
class RBM():
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv) # initial all weights
        self.a = torch.randn(1, nh) # initialize all hidden nodes
        self.b = torch.randn(1, nv) # initialize all visible nodes
    def sample_h(self, x):
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx) # activation function
        p_h_given_v = torch.sigmoid(activation) # probablity hidden node is activated given the visible node
        return p_h_given_v, torch.bernoulli(p_h_given_v) # return also the sampling of the hidden nodes
    def sample_v(self, y):
        wy = torch.mm(y, self.W)
        activation = wy + self.a.expand_as(wy) # activation function
        p_v_given_h = torch.sigmoid(activation) # probablity visible node is activated given the hidden node
        return p_v_given_h, torch.bernoulli(p_v_given_h) # return also the sampling of the visible nodes
    def train(self, v0, vk, ph0, phk):
        self.W += torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)





















